Importimine

In [3]:
import zipfile
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models

Kopsupõletikku tuvastamine

In [4]:
# Path to the ZIP file
zip_path = "C:/Users/kasnikov/Desktop/All about coding/Tehisintellekt/archive.zip"
extract_path = "C:/Users/kasnikov/Desktop/All about coding/Tehisintellekt/extracted_photos"

# Extract the ZIP file
with zipfile.ZipFile(zip_path, "r") as zip_file:
    zip_file.extractall(extract_path)


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/kasnikov/Desktop/All about coding/Tehisintellekt/archive.zip'

In [ ]:
train_dir = os.path.join(extract_path, 'chest_xray/chest_xray/train')
test_dir = os.path.join(extract_path, 'chest_xray/chest_xray/test')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)



In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



history = model.fit(
    train_generator,
    steps_per_epoch=100,  # Number of batches to yield from the generator per epoch
    epochs=1,
    batch_size = 512,
    validation_data=test_generator,
    validation_steps=50   # Number of batches to yield from the generator for validation
)



test_loss, test_acc = model.evaluate(test_generator, steps=50)
print(f"Test accuracy: {test_acc}")

In [ ]:
def load_and_preprocess_image(img_path, target_size=(150, 150)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale to 0-1 range
    return img_array